In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install dreamsboard -U

### 介绍
使用langchain进行任务规划，构建子任务的会话场景资源，通过MCTS任务执行器，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


## 使用

### 构建任务

- 初始化任务引擎 StructuredTaskStepStoryboard传入需要的任务

- loader_task_step_iter_builder 构建任务的子任务，完成后SimpleTaskStepStore可获取子任务信息

- init_task_engine_dreams 初始化场景加载资源，对子任务进行规划，获取会话的资源信息

```

    os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
    os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
    os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
    os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
    os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
    os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST
```

- init_task_engine_storyboard_executor 构建会话场景执行器，初始化一个会话

- storyboard_code_gen_builder 构建会话场景执行器, 对会话存储进行加载，加载失败重新构建

- generate_step_answer 通过会话场景 获取任务的答案





```

In [4]:
   
from dreamsboard.engine.storage.task_step_store.types import BaseTaskStepStore
from dreamsboard.engine.task_engine_builder.core import TaskEngineBuilder
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore
from langchain_community.chat_models import ChatOpenAI
from dreamsboard.dreams.builder_task_step.base import StructuredTaskStepStoryboard
from dreamsboard.engine.utils import concat_dirs
from dreamsboard.engine.storage.task_step_store.types import DEFAULT_PERSIST_FNAME
from dreamsboard.common.try_parse_json_object import try_parse_json_object
from dreamsboard.engine.memory.mctsr.prompt import RefineResponse
from dreamsboard.dreams.task_step_md.base import TaskStepMD
from dreamsboard.common import _get_assistants_tool

import logging
import threading
import os
from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 控制台打印
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

logger.addHandler(handler)

In [14]:

llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
kor_dreams_task_step_llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.95,
    top_p=0.70,
)

deepseek_llm = ChatOpenAI(
    openai_api_base=os.environ.get("DEEPSEEK_API_BASE"),
    model=os.environ.get("DEEPSEEK_API_MODEL"),
    openai_api_key=os.environ.get("DEEPSEEK_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
zhipuai_llm = ChatOpenAI(
    openai_api_base=os.environ.get("ZHIPUAI_API_BASE"),
    model=os.environ.get("ZHIPUAI_API_MODEL"),
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
if 'glm' in os.environ.get("API_MODEL"):

    tools= [ { "type": "web_search",   "web_search": {"enable": False ,"search_result": False   }}]
else:
    tools = []
llm_with_tools = llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )
kor_dreams_task_step_llm_with_tools = kor_dreams_task_step_llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )

from tests.test_builder_task_step.prompts import (
    AEMO_REPRESENTATION_PROMPT_TEMPLATE as AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST,
    STORY_BOARD_SCENE_TEMPLATE as STORY_BOARD_SCENE_TEMPLATE_TEST,
    STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE as STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST,
    EDREAMS_EVOLUTIONARY_TEMPLATE as EDREAMS_EVOLUTIONARY_TEMPLATE_TEST,
    EDREAMS_PERSONALITY_TEMPLATE as EDREAMS_PERSONALITY_TEMPLATE_TEST,
    DREAMS_GEN_TEMPLATE as DREAMS_GEN_TEMPLATE_TEST,
) 
os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST


# 存储
cross_encoder_path = "/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
embed_model_path = "/mnt/ceph/develop/jiawei/model_checkpoint/m3e-base"
start_task_context = "什么是损失函数？"
builder = StructuredTaskStepStoryboard.form_builder(
    llm_runable=llm_with_tools,
    kor_dreams_task_step_llm=kor_dreams_task_step_llm_with_tools,
    start_task_context=start_task_context,
    cross_encoder_path=cross_encoder_path,
    embed_model_path=embed_model_path
)

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not instal

In [15]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=True)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

2e3a7c8d-7783-48f5-8344-79a0816bf207
c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d
ebde806e-4129-49b2-ae14-a53024b4da31
94b00ac0-f040-4b86-a4cf-d0a341d92cd8
0664ac70-89c3-481c-976f-ae120a264ff4
86c7af97-ccb6-4a5a-bbe5-d86ce755bac4
0fe8a867-be0a-4937-aa37-5a97fb432a3c


In [16]:

from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./60f9b7459a7749597e7efa71d1747bc4/storage")
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,2e3a7c8d-7783-48f5-8344-79a0816bf207,1,story_board0,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,损失函数的定义,损失函数（Loss Function）是一个用于衡量模型预测结果与真实标签之间差异的函数。在...,0,,[],,
1,c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d,2,story_board1,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,分析近几年研究领域中损失函数相关的技术框架与方法论,在不同研究领域，损失函数的设计与选择紧密关联技术框架与方法论。例如在深度学习框架下，根据任务...,0>1,,[],,
2,ebde806e-4129-49b2-ae14-a53024b4da31,3,story_board2,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,研究论文中损失函数在不同任务中的应用与变体,分类任务：交叉熵损失函数是主流选择，如在图像分类的卷积神经网络（CNN）中广泛应用。为应对类...,0>2,,[],,
3,94b00ac0-f040-4b86-a4cf-d0a341d92cd8,4,story_board3,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,评估学术界损失函数相关的技术进步与局限性,技术进步：近年来损失函数的设计不断创新，针对不同任务和数据特点开发出多种有效的变体，显著提升...,0>3,,[],,
4,0664ac70-89c3-481c-976f-ae120a264ff4,5,story_board4,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,探讨损失函数在不同数据集与应用场景下的适用性与泛化能力,在不同数据集上，损失函数的表现差异较大。对于大规模、高质量且分布均匀的数据集，标准的损失函数...,0>4,,[],,
5,86c7af97-ccb6-4a5a-bbe5-d86ce755bac4,6,story_board5,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,分析最新损失函数算法的稳定性与容错性,一些最新的损失函数算法针对稳定性和容错性进行了优化。例如在复杂动态环境下，部分自适应损失函数...,0>5,,[],,
6,0fe8a867-be0a-4937-aa37-5a97fb432a3c,7,story_board6,什么是损失函数？,### 损失函数的定义\n损失函数（Loss Function）是一个用于衡量模型预测结果与...,评估论文中提出的损失函数相关未来研究方向与挑战,新研究问题：提出了如何设计通用的、不依赖特定任务和数据分布假设的损失函数，以提高模型在不同场...,0>6,,[],,




#### 场景加载模块设计方案
```

编写符合计算机科学领域的 故事情境提示词，生成研究情境（story_scenario_context），替换现有的langchain会话模板，
1、对这个提示词所要求的输入拆分成子任务，
2、对每个子任务指令转换为子问题，召回问题前3条，
3、对召回内容与问题拼接，合成会话内容变量（scene_content）


对每个子问题相关的召回内容，转换为第一人称的会话总结（研究场景（scene_monologue_context）），

1、对召回内容与问题拼接，对每个前3条组成一个总结任务的提示词，为每个任务标记唯一编号，组成任务上下文（story_board_summary_context）
2、加载编号和story_board_summary_context，转换会话信息
```



#### MCTS设计方案

```
MCT 自优化算法代表了蒙特卡洛树搜索（MCTS）与大型语言模型的结合，将不同的场景执行任务过程抽象为搜索树结构。树上的节点代表当前不同视角下的选择策略，而边表示主体对自身的反思。该算法的操作流程遵循 MCTS 算法的一般模式。

具体来说，我们采用模型的先验知识，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


1、对问题生成的子任务，生成一个合理的规划的节点
2、对每个节点创建一个MCTS任务，
3、输入 problem（总问题的子任务相关的子问题）
4、评分代码重构，将片段摘录器模块集成到一个关于_evaluate_answer逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于evaluate_system_prompt 的生成策略，具体的为编写一个关于带有评估的评估器，由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message

5、自我反思代码重构,将片段摘录器模块集成到一个关于self_refine逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于critic_system_prompt和refine_system_prompt的生成策略，critic_system_prompt为生成一个关于子问题相关的loader_cosplay_builder中自身不完美的评价内容，refine_system_prompt为不完美评价的思考过程和评分值。
具体的为编写一个关于带有评价的生成器和反思生成器，它们由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message
```



#### MCTS执行中的重要环节
```
MCTS中的约束规则如下，需要保证这些节点必须符合下面所定义的基本规则

提示约束：模型在奖励评分期间必须遵守最严格的标准。生成结果需要为JSON Response format
{
    "thought": "The thought process behind the answer.",
    "answer": "A float representing the answer to the problem."
}


高分抑制：评分节点中不存在满分反馈机制；任何超过 95 分的奖励都会按固定金额减少，以遏制过高分数。

重复采样：每次访问搜索树节点都涉及对节点奖励的重复采样，以增强自我评估的可靠性。需要注意的是，当对节点的子节点进行奖励采样时，我们也会对其父节点进行奖励采样，以增加奖励采样的样本量。
```



In [17]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

2e3a7c8d-7783-48f5-8344-79a0816bf207
c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d
ebde806e-4129-49b2-ae14-a53024b4da31
94b00ac0-f040-4b86-a4cf-d0a341d92cd8
0664ac70-89c3-481c-976f-ae120a264ff4
86c7af97-ccb6-4a5a-bbe5-d86ce755bac4
0fe8a867-be0a-4937-aa37-5a97fb432a3c


In [18]:
 


def worker(step: int, task_engine: TaskEngineBuilder, task_step_store: BaseTaskStepStore):
    owner = f"step:{step}, task_step_id:{task_engine.task_step_id}, thread {threading.get_native_id()}"
    logger.info(f"{owner}，任务开始")
    if not task_engine.check_engine_init():
        task_engine.init_task_engine()
        task_engine.init_task_engine_dreams()
        task_engine.init_task_engine_storyboard_executor()

    try:
        logger.info(f"{owner}，storyboard_code_gen_builder")
        code_gen_builder = task_engine.storyboard_code_gen_builder()
        task_step = task_engine.task_step_store.get_task_step(task_engine.task_step_id)
        if task_step.task_step_question_answer is None or len(task_step.task_step_question_answer) == 0:
            task_engine.generate_step_answer(code_gen_builder)
        
        logger.info(f"step:{step}, {owner}，get_mcts_node")
        mcts_node = task_engine.get_mcts_node()
        if step % 2 == 0:
            mcts_node.llm_runable = deepseek_llm
        if step % 3 == 0:
            mcts_node.llm_runable = zhipuai_llm
        logger.info(f"step:{step}, {owner}，get_mcts_node run")
        answer = mcts_node.run()
        
        mcts_node.print()
        print(answer)
        task_step.task_step_question_answer = answer 
        task_step_id = task_engine.task_step_id
        
        task_engine.task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage/{task_step_id}", basename=DEFAULT_PERSIST_FNAME)
        task_engine.task_step_store.persist(persist_path=task_step_store_path) 
        
        task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage", basename=DEFAULT_PERSIST_FNAME)
        task_step_store.persist(persist_path=task_step_store_path) 

    except Exception as e:
        logger.error("场景加载失败", e)

    logger.info(f"{owner}，任务结束")




if __name__ == "__main__":
    threads = []
    
    step =0
    task_step_store = builder.task_step_store
    # 初始化任务引擎
    task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
    while not task_engine_builder.empty():
       
        task_engine = task_engine_builder.get()
        step += 1
        t = threading.Thread(target=worker,
                             kwargs={"step": step, "task_engine": task_engine, "task_step_store": task_step_store},
                             daemon=True)
        t.start()
        threads.append(t) 
    
    for t in threads:
        t.join()

step:1, task_step_id:2e3a7c8d-7783-48f5-8344-79a0816bf207, thread 258019，任务开始
step:2, task_step_id:c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d, thread 258020，任务开始
step:3, task_step_id:ebde806e-4129-49b2-ae14-a53024b4da31, thread 258021，任务开始
step:4, task_step_id:94b00ac0-f040-4b86-a4cf-d0a341d92cd8, thread 258022，任务开始
step:5, task_step_id:0664ac70-89c3-481c-976f-ae120a264ff4, thread 258023，任务开始
step:6, task_step_id:86c7af97-ccb6-4a5a-bbe5-d86ce755bac4, thread 258024，任务开始
step:7, task_step_id:0fe8a867-be0a-4937-aa37-5a97fb432a3c, thread 258025，任务开始
Searching papers for query: 学术界在损失函数相关方面取得了哪些技术进步，又存在哪些局限性？ 
Found 30 papers.
Found 30 unique paper IDs.
Fetching details for paper ID: 628afb4c5aee126c0f04e3e8
Searching papers for query: 在小样本、不均衡或存在噪声的数据集上，有哪些具体的针对性损失函数变体可以有效提升模型性能？ 
Searching papers for query: 在研究论文中，损失函数在分类、回归和生成任务中分别有哪些典型应用及变体，这些应用和变体是如何针对不同任务特点来提升模型性能的？ 
Fetching details for paper ID: 658b928f939a5f40825cf053
Searching papers for query: 在极端复杂、动态变化剧烈的环境中，除了实时对抗性环境会使损失函数剧烈波动影响训练和

Iteration complete for resource_collection_94b00ac0-f040-4b86-a4cf-d0a341d92cd8.


Fetching details for paper ID: 64671268d68f896efaf14a15
Data cached for query: 论文中提出的损失函数相关未来研究方向里，结合元学习、强化学习设计动态自适应损失函数时，具体会面临哪些技术难点来实现根据模型学习过程和环境反馈实时调整损失函数的形式和参数 ？  at /tmp/query_cache/61daa058af012d45485b169caeae1fe6.json
owner:register_event thread 258025
owner:getlock thread 259561, resource_id:resource_collection_0fe8a867-be0a-4937-aa37-5a97fb432a3c
owner:lock thread 259561, resource_id:resource_collection_0fe8a867-be0a-4937-aa37-5a97fb432a3c
thread 259561 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259561 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259561 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
thread 259561 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/common/callback.py", line 71, in _run
    self.mfunc(callback=self._callback, resource_id=self.resource_id, **self.kwargs)
  File "/mnt/ceph/develop/jiaw

Iteration complete for resource_collection_0fe8a867-be0a-4937-aa37-5a97fb432a3c.


owner:getlock thread 259606, resource_id:resource_collection_2e3a7c8d-7783-48f5-8344-79a0816bf207
owner:lock thread 259606, resource_id:resource_collection_2e3a7c8d-7783-48f5-8344-79a0816bf207
thread 259606 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259606 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259606 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
thread 259606 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/common/callback.py", line 71, in _run
    self.mfunc(callback=self._callback, resource_id=self.resource_id, **self.kwargs)
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/dreams/task_step_to_question_chain/base.py", line 189, in _into_database_query
    kwargs.get("collection").do_add_doc(docs)
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard

...

Iteration complete for resource_collection_ebde806e-4129-49b2-ae14-a53024b4da31.


Fetching details for paper ID: 65cd71a5939a5f40823651e7
Data cached for query: 在深度学习框架下，除了图像分类任务常用交叉熵损失函数、回归任务常用均方误差损失函数外，其他不同类型的任务（如目标检测、语义分割等）分别适合选择哪些损失函数，其选择依据是什么？  at /tmp/query_cache/76e6e4a05cda5a64f42893395024cd43.json
owner:register_event thread 258020
owner:getlock thread 259801, resource_id:resource_collection_c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d
owner:lock thread 259801, resource_id:resource_collection_c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d
thread 259801 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259801 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259801 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
thread 259801 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/common/callback.py", line 71, in _run
    self.mfunc(callback=self._callback, resource_id=self.resource_id, **self.kwargs)
  File "/mnt/ceph/develop/jia

Iteration complete for resource_collection_c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d.


owner:getlock thread 259846, resource_id:resource_collection_86c7af97-ccb6-4a5a-bbe5-d86ce755bac4
owner:lock thread 259846, resource_id:resource_collection_86c7af97-ccb6-4a5a-bbe5-d86ce755bac4
thread 259846 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259846 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。
thread 259846 开始操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
thread 259846 结束操作：('60f9b7459a7749597e7efa71d1747bc4', 'samples')。插入
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/common/callback.py", line 71, in _run
    self.mfunc(callback=self._callback, resource_id=self.resource_id, **self.kwargs)
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/dreams/task_step_to_question_chain/base.py", line 189, in _into_database_query
    kwargs.get("collection").do_add_doc(docs)
  File "/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard

Iteration complete for resource_collection_86c7af97-ccb6-4a5a-bbe5-d86ce755bac4.


In [19]:

store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./60f9b7459a7749597e7efa71d1747bc4/storage")
task_step_md = TaskStepMD(store_load)
md_text =   task_step_md.format_md() 
from IPython.display import Markdown, display

display(Markdown(md_text.text))


# 什么是损失函数？ 


### 损失函数的定义 [task_id:0](2e3a7c8d-7783-48f5-8344-79a0816bf207)

损失函数（Loss Function）是一个用于衡量模型预测结果与真实标签之间差异的函数。在机器学习和深度学习中，模型的目标通常是最小化这个损失函数，以便使模型的预测尽可能接近真实值。

分析近几年研究领域中损失函数相关的技术框架与方法论 [task_id:0>1](c7cd45bd-b59c-49cc-b0b1-f5253f5f5a9d)在不同研究领域，损失函数的设计与选择紧密关联技术框架与方法论。例如在深度学习框架下，根据任务类型（分类、回归等）选择合适的损失函数。如在图像分类任务中常用交叉熵损失函数，它基于概率分布的差异度量，符合分类任务预测类别概率的特性；在回归任务里，均方误差损失函数较为常用，通过计算预测值与真实值差值的平方来衡量误差，反映预测的偏离程度。不同方法论也会影响损失函数的构建，如强化学习中基于策略梯度的方法，其损失函数设计围绕最大化累计奖励，与监督学习中的损失函数设计思路有明显区别。

研究论文中损失函数在不同任务中的应用与变体 [task_id:0>2](ebde806e-4129-49b2-ae14-a53024b4da31)分类任务：交叉熵损失函数是主流选择，如在图像分类的卷积神经网络（CNN）中广泛应用。为应对类别不均衡问题，出现了变体如焦点损失（Focal Loss），它通过对不同类别样本赋予不同权重，加大对难分类样本的学习权重，提升模型在不均衡数据上的分类性能。在文本分类任务中，除了标准交叉熵损失，还会结合对抗训练引入对抗损失，以增强模型对文本语义的理解和泛化能力。回归任务：均方误差损失函数是基础，但在一些对异常值敏感的场景下，会采用平均绝对误差（MAE）损失函数，它对异常值的鲁棒性更强。还有 Huber 损失函数，结合了 MSE 和 MAE 的优点，在误差较小时类似 MSE 以加快收敛，误差较大时类似 MAE 增强鲁棒性。生成任务：生成对抗网络（GAN）中，生成器和判别器的训练基于对抗损失。生成器试图最小化生成样本与真实样本在判别器上的差异损失，判别器则最大化区分真实样本和生成样本的损失。此外，在变分自编码器（VAE）中，使用重构损失和 KL 散度损失共同约束模型学习数据的潜在表示和重构能力。

评估学术界损失函数相关的技术进步与局限性 [task_id:0>3](94b00ac0-f040-4b86-a4cf-d0a341d92cd8)技术进步：近年来损失函数的设计不断创新，针对不同任务和数据特点开发出多种有效的变体，显著提升了模型性能。例如上述提到的焦点损失、对抗损失等，在实际应用中取得了良好效果。同时，随着理论研究的深入，对损失函数的性质和优化过程有了更清晰的理解，有助于更合理地选择和设计损失函数。局限性：尽管取得进步，但损失函数仍存在一些问题。一方面，很多损失函数的设计依赖于特定的任务假设和数据分布，泛化能力有限。例如某些针对特定数据集设计的损失函数变体，在其他数据集上效果不佳。另一方面，模型偏差和数据依赖问题依然存在。如在小样本数据上，损失函数可能无法充分学习到数据的特征，导致模型过拟合；在存在噪声的数据中，损失函数可能对噪声敏感，影响模型性能。

探讨损失函数在不同数据集与应用场景下的适用性与泛化能力 [task_id:0>4](0664ac70-89c3-481c-976f-ae120a264ff4)在不同数据集上，损失函数的表现差异较大。对于大规模、高质量且分布均匀的数据集，标准的损失函数如交叉熵损失、均方误差损失通常能取得较好效果。但在小样本、不均衡或存在噪声的数据集上，需要针对性的损失函数变体来提升模型性能。在应用场景方面，医疗影像诊断中，由于数据稀缺且对诊断准确性要求高，需要设计能充分利用有限数据且对微小病变敏感的损失函数；在自动驾驶场景下，数据具有动态性和实时性，损失函数要能适应环境变化，保证模型在复杂路况下的稳定性和准确性。然而，目前很多损失函数在跨领域、跨模态数据上的泛化能力仍有待提高，难以直接应用于多领域融合的复杂场景。

分析最新损失函数算法的稳定性与容错性 [task_id:0>5](86c7af97-ccb6-4a5a-bbe5-d86ce755bac4)一些最新的损失函数算法针对稳定性和容错性进行了优化。例如在复杂动态环境下，部分自适应损失函数能够根据训练过程中的数据变化自动调整参数，保持模型训练的稳定性。在大规模数据上，一些分布式训练的损失函数算法通过优化通信和计算策略，提高算法的适应性和效率。但总体而言，在极端复杂、动态变化剧烈的环境中，损失函数算法的稳定性和容错性仍面临挑战，例如在实时对抗性环境中，模型可能因环境突变导致损失函数剧烈波动，影响训练和预测效果。

评估论文中提出的损失函数相关未来研究方向与挑战 [task_id:0>6](0fe8a867-be0a-4937-aa37-5a97fb432a3c)新研究问题：提出了如何设计通用的、不依赖特定任务和数据分布假设的损失函数，以提高模型在不同场景下的泛化能力。还关注如何利用无监督或弱监督数据设计损失函数，降低对大规模标注数据的依赖。现有问题改进：针对模型偏差和数据依赖问题，研究如何通过损失函数的设计使模型更好地学习数据的本质特征，减少过拟合和欠拟合现象。同时，致力于提升损失函数在复杂环境下的稳定性和容错性，确保模型在各种干扰和变化下仍能正常工作。新研究切入点：结合新的技术如元学习、强化学习来设计动态自适应的损失函数，根据模型的学习过程和环境反馈实时调整损失函数的形式和参数。这些挑战推动后续研究不断探索新的理论和方法，以改进损失函数的设计和应用，提升模型的整体性能和适应性。


 



StructuredTaskStepStoryboard
    
对任务进行规划，生成段落之间组成一个动态上下文，  扩写任务步骤构建MCTS任务
    输入：
    	start_task_context： 起始任务
    	llm： 模型
    	kor_dreams_task_step_llm: 任务抽取模型
    	task_step_store: 任务存储器（SimpleTaskStepStore）
    	cross_encoder_path: ranking模型路径（sentence_transformers），当前设计耦合了业务，之后会单独设计一个召回模块

    任务：

        1、对任务（AEMO_REPRESENTATION_PROMPT_TEMPLATE）按照提示词要求进行扩写，将扩写任务步骤收集 （src/dreamsboard/dreamsboard/engine/entity/task_step、src/dreamsboard/tests/test_kor/test_kor3.py）

        2、收集每个任务后存储到磁盘（src/dreamsboard/dreamsboard/engine/storage/task_step_store）

        3、对每个子任务载入会话场景，然后按照扩写任务步骤构建，MCTS任务 loader_task_step_iter_builder


	》 TaskEngineBuilder 场景加载模块
		执行会话场景资源初始化，构建MCTS任务

    根据任务步骤，构建场景加载模块，生成资源文件csv
    根据每个任务，载入StructuredDreamsStoryboard 会话场景
    按照扩写任务步骤构建MCTS任务

		输入：
			task_step_id
			task_step_store: 任务存储器（SimpleTaskStepStore）
			start_task_context： 起始任务
			llm： 模型

		任务：
			init_task_engine：
				初始化任务引擎
        》TaskStepToQuestionChain 
        	输入：
        		client： 矢量库客户端
        		llm： 模型
					invoke_task_step_to_question：1、 对开始任务进行抽取，得到任务步骤，提示词所要求的输入拆分成子任务， 
					invoke_task_step_question_context： 2、对每个子任务指令转换为子问题，召回问题前3条，对任务步骤进行抽取，得到任务步骤的上下文
					export_csv_file_path: 3、对召回内容与问题 导出csv文件

			init_task_engine_dreams
				初始化场景加载资源
					StoryBoardDreamsGenerationChain
					对每个子任务通过职业提示词，载入会话场景
						1、构建场景信息（story_scenario_context），提示词（STORY_BOARD_SCENE_TEMPLATE）
						2、对任务上下文(story_board_summary_context)，构建第一人称数据(scene_monologue_context),提示词（STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE）
						3、对任务上下文(story_board_summary_context)，获取任务分析(evolutionary_step), 提示词（EDREAMS_EVOLUTIONARY_TEMPLATE）
						4、对任务分析(evolutionary_step)，分析对话预设信息（性格）， 提示词（EDREAMS_PERSONALITY_TEMPLATE）
						5、对任务上下文(story_board_summary_context)，场景信息story_scenario_context, 第一人称数据(scene_monologue_context)，
						生成关于人物职业的引导话术，提示词（DREAMS_GEN_TEMPLATE）

			init_task_engine_storyboard_executor

				构建会话场景执行器 StructuredDreamsStoryboard
					对剧本和分析结果进行结构化，将开放问题与性格分析结果进行结合。生成情景扮演会话场景执行器
			    此过程如下
			        对开放问题结果进行抽取，得到问题内容
			        对性格分析结果进行抽取，得到性格分析结果
			        增加系统提示词
			        根据剧本与任务性格基础情景扮演代码，根据每步的抽取析得到的问题，生成问题的答案
			       	在上下文中增加，关于人物职业的引导话术
	        		导出会话场景执行器
					
	    storyboard_code_gen_builder
	    	构建会话场景执行器

	    generate_step_answer
	    	获取主进程子任务的答案
	    
	    get_mcts_node
	    	构建MCTS树, 初始化当前任务相关的MCTS节点，并返回MCTS执行器
